<a href="https://colab.research.google.com/github/cs-pub-ro/ML/blob/master/lab2/Laborator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Învățare Automată
# Grupare ierarhică.
### Autori:
* Tudor Berariu - 2016
* George Muraru - 2020

## 1. Scopul laboratorului
Scopul laboratorului îl reprezintă înțelegerea grupării ierarhice și a diferențelor dintre aceasta și algoritmul K-Means prezentat în primul laborator.

## 2. Introducere
În primul laborator a fost prezentat un algoritm pentru grupare, K-Means, care suferea de câteva limitări importante (soluția depindea de alegerea centroizilor inițiali; numărul de grupuri trebuie cunoscut sau intuit).

De asemenea, s-a observat faptul că anumite seturi de date nu pot fi grupate folosind algoritmul K-Means din cauza *formei* grupurilor. Pentru a rezolva problema grupării seturi de date este nevoie de o abordare diferită de cea a reprezentării grupurilor prin centroizi.

O altă abordare a problmei grupării unei mulțimi de obiecte în funcție de similaritatea dintre acestea (engl. *cluster analysis*) o reprezintă **gruparea ierarhică** (engl. *hierarchical clustering*).

Spre deosebire de algoritmul K-Means, în cazul grupării ierarhice nu este necesară stabilirea a priori a numărului de grupuri și a unei partiționări inițiale a obiectelor.

Gruparea ierarhică are două variante:
* **grupare aglomerativă** (agglomerative clustering) - în care se pornește de la situația în care fiecare obiect formează singur un grup. Apoi se reunesc succesiv cele mai apropiate două grupuri până când
rămâne unul singur.

* **grupare prin divizare** - în care se pornește de la un singur grup ce cuprinde toate obiectele, iar la fiecare pas se alege un grup (cel mai eterogen pentru a fi segmentat.

Rezultatul produs de gruparea ierarhică este un arbore de grupări / divizări succesive. De obicei, acest arbore este reprezentat grafic, pentru proporții ținându-se cont de similaritatea inter-cluster, printr-o [dendrogramă](#test). Numărul de grupuri potrivit pentru problemă se alege la final, de cele mai multe ori după vizualizarea dendrogramei.


<a name="test"><img src="https://drive.google.com/uc?export=view&id=1y0k0IWywdC_3ZNRtNjfqr2UT5O7PkwIz" alt="Dendogram image"/></a>

<p align="center">
Figura 1: <a href="http://www.macs.hw.ac.uk/texturelab/people/thomas-methven#Surface">Exemplu de dendogramă</a>
</p>

## 3. Măsurarea apropierii dintre două grupuri
Gruparea ierarhică reprezintă, de fapt, o familie de algoritmi ce folosesc diferite definiții ale distanței (similarității) dintre obiecte pentru construirea clusterelor.

* **single-linkage** - distanța (similaritatea) dintre cele mai apropiate două puncte
$$ D_{SL}(G, H) = \min_{i \in G, j \in H} d_{i,j} $$

* **complete-linkage** (**maximum-linkage**) - distanța (similaritatea) dintre cele mai depărtate două puncte
$$ D_{CL}(G, H) = \max_{i \in G, j \in H} d_{i,j} $$

* **group-average** - distanța (similaritatea) medie a celor două grupuri
$$ D_{GA}(G, H) = \frac{1}{|G||H|}\sum_{i \in G}\sum_{j \in H} d_{i,j} $$

* **ward-linkage** - distanța (similaritatea) dintre 2 grupări este dată de creșterea distanței față de (noul) centroid dacă cele 2 grupări s-ar uni
$$ D_{WL}(G, H) = \sum_{i \in G \cup H}d_{i, u_{GH}}^2 - \sum_{i \in G}d_{i, u_{G}}^2 - \sum_{i \in H}d_{i, u_{H}}^2 $$

# 4. Alte metode de clusterizare
În afara metodelor bazate pe centroid (K-Means) și pe conectivitate (grupare ierarhică), alte (câteva) modele de grupare sunt:
* metode bazate pe distribuții - ex: [GMM](https://scikit-learn.org/stable/modules/mixture.html)
* metode bazate pe densitate - ex: [DBSCAN](https://www.coursera.org/lecture/predictive-analytics/dbscan-EVHfy)
* metode bazate pe teoria grafurilor - [bazat pe MST (Minimum Spanning Tree)](https://www.cs.cmu.edu/~ckingsf/bioinfo-lectures/mst.pdf)

O lectură completă cu metodele de clusterizare poate fi găsită la [[1]](#CL).

# 5. Workspace Setup

### Dependențe


In [0]:
!pip install numpy
!pip install scipy

### Câteva biblioteci de care vom avea nevoie

In [0]:
%matplotlib inline

import numpy as np
from zipfile import ZipFile

# Clustering
from scipy.cluster import hierarchy

# Plotting stuff
import matplotlib.pyplot as plt
import matplotlib.markers
from mpl_toolkits.mplot3d import Axes3D

### Parametrii necesari rulării

In [0]:
DATASET_NAME = 'Atom'  #@param ['Atom', 'Chainlink', 'EngyTime', 'GolfBall', 'Hepta', 'Lsun', 'Target', 'TwoDiamonds', 'WingNut']

### Funcții ajutătoare pentru descărcarea și lucrul cu setul de date

In [0]:
def dummy(Xs):
    """ Creates a dummy array for the hierarchical associative clustering
   
    Args:
        Xs (numpy array): dataset
    Returns:
        A numpy array with size (nr_datapoints - 1) x 4
          where an entry in the array looks like:
            id_c1, id_c2, dist_c1c2, nr_datapoints_c1c2
    """

    (N, D) = Xs.shape
    Z = np.zeros((N-1, 4))
    lastIndex = 0

    for i in range(N-1):
        Z[i,0] = lastIndex
        Z[i,1] = i+1
        Z[i,2] = 0.1 + i
        Z[i,3] = i+2
        lastIndex = N+i
    return Z
 

def getArchive():
    """ Checks if FCPS.zip is present in the local directory, if not,
    downloads it.

    Returns:
        A ZipFile object for the FCPS archive
    """

    archive_url = ("https://github.com/cs-pub-ro/ML/raw/master/lab1/FCPS.zip")
    local_archive = "FCPS.zip"
 
    from os import path
    if not path.isfile(local_archive):
        import urllib
        print("Downloading...")
        urllib.request.urlretrieve(archive_url, filename=local_archive)
        assert path.isfile(local_archive)
        print("Got the archive")

    return ZipFile(local_archive)


def getDataSet(archive, dataSetName):
    """ Get a dataset from the FCPS.zip

    Args:
        archive (ZipFile): Object for the FCPS
        dataSetName (String): The dataset name from the FCPS

    Returns:
        A tuple (Xs, labels)
        Xs (numpy array): rows are the elements and the cols are the features
        labels (numpy array): labels associated with Xs

    """

    path = "FCPS/01FCPSdata/" + dataSetName
 
    lrnFile = path + ".lrn"
    with archive.open(lrnFile, "r") as f:
        N = int(f.readline().decode("UTF-8").split()[1])
        D = int(f.readline().decode("UTF-8").split()[1])
        f.readline()
        f.readline()
        Xs = np.zeros([N, D-1])
        for i in range(N):
            data = f.readline().decode("UTF-8").strip().split("\t")
            assert len(data) == D
            assert int(data[0]) == (i + 1)
            Xs[i] = np.array(list(map(float, data[1:])))

    clsFile = path + ".cls"
    with archive.open(clsFile, "r") as f:
        labels = np.zeros(N).astype("uint")
 
        line = f.readline().decode("UTF-8")
        while line.startswith("%"): 
            line = f.readline().decode("UTF-8")
 
        i = 0
        while line and i < N:
            data = line.strip().split("\t")
            assert len(data) == 2
            assert int(data[0]) == (i + 1)
            labels[i] = int(data[1])
            line = f.readline().decode("UTF-8")
            i = i + 1
 
        assert i == N
 
    return Xs, labels


def plotClusters(Xs, labels, K, clusters):
    """ Plot the data with the true labels alongside the centroids and the
    predicted cluster.
    If the elements from the dataset are not 2 or 3 dimensional then print
    the index, predicted cluster and true label.

    Args:
        Xs (numpy array): dataset
        labels (numpy array): real/true labels
        K (int): number of clusters
        clusters (numpy array): predicted labels
    """

    labelsNo = np.max(labels)

    markers = []

    while len(markers) < labelsNo:
        markers.extend(list(matplotlib.markers.MarkerStyle.filled_markers))

    colors = plt.cm.rainbow(np.linspace(0, 1, K+1))
    if Xs.shape[1] == 2:
        x = Xs[:,0]
        y = Xs[:,1]
        for (_x, _y, _c, _l) in zip(x, y, clusters, labels):
            plt.scatter(_x, _y, s=500, c=[colors[_c]], marker=markers[_l])
        plt.show()
    elif Xs.shape[1] == 3:
        x = Xs[:,0]
        y = Xs[:,1]
        z = Xs[:,2]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        for (_x, _y, _z, _c, _l) in zip(x, y, z, clusters, labels):
            ax.scatter(_x, _y, _z, s=200, c=[colors[_c]], marker=markers[_l])
        plt.show()
    else:
        for i in range(Xs.shape[0]):
            print(f"{i} : {clusters[i]} ~ {labels[i]}")

### Încărcare set de date

In [0]:
Xs, labels = getDataSet(getArchive(), DATASET_NAME)

## 5. Cerințe

1. [4 pct] Implementați algoritmul de grupare ierarhică aglomerativă folosind distanța [*single-linkage*](#scrollTo=2-gZRj6ZMe6z).
    
    Funcția trebuie să ı̂ntoarcă o matrice cu $(N − 1) × 4$ valori.
    
    Fiecare linie corespunde unei alipiri a două grupuri (plecând de la N grupuri se ajunge la unul singur ı̂n N − 1 pași).
  * primele două valori corespund id-urilor grupurilor ce trebuie unite
  * a treia valoare conține distanța dintre cele două grupuri
  * a patra valoare reprezintă numărul de puncte pe care le conține noul grup

  Pentru id-uri, valorile de la 0 la $N − 1$ se referă la exemplele din setul de date, iar cele de la $N$ la $2N − 2$ corespund grupurilor construite pe parcurs (la pasul $ 0 \le i < N −1 $ se construiește clusterul $N+i$).

In [0]:
def singleLinkage(Xs):
     # TODO 1
     return dummy(Xs)

Z = singleLinkage(Xs)
dn = hierarchy.dendrogram(Z)


2. [2 pct] Implementați distanțele:
  * *complete-linkage*
  * *group-average*.

    Cele două funcții ı̂ntorc o matrice cu aceeași semantică precum ı̂n cazul *single-linkage*.

In [0]:
def completeLinkage(Xs):
    # TODO 2
    return dummy(Xs)

In [0]:
def groupAverageLinkage(Xs):
    # TODO 2
    return dummy(Xs)

3. [2 pct] Implementați funcția *extractClusters* care pe baza unei aglomerări ierarhice construite anterior, stabilește numărul optim de clustere ca fiind cel dinaintea alipirii făcute la cea mai mare distanță.

In [0]:
def extractClusters(Xs, Z):
    (N, D) = Xs.shape
    assert (Z.shape == (N-1, 4))

    #TODO 3
    return 1, np.zeros(N, dtype=np.uint64)

K, clusters = extractClusters(Xs, Z)
plotClusters(Xs, labels, K, clusters)


4. [2 pct] Testați algoritmul implementat și eficiența acestuia pe seturile de date din arhivă.

## 6. Set de date
În cadrul acestui laborator veți folosi seturile de date [FCPS](https://github.com/cs-pub-ro/ML/raw/master/lab1/FCPS.zip) (Fundamental Clustering
Problem Suite) ale Philipps Universität Marburg.

Pentru fiecare set de date veți găsi următoarele fișiere ı̂n subdirectorul 01FCPSdata:
* $<$nume$>$.lrn - setul de date cu un id pentru fiecare obiect,
* $<$nume$>$.cls - clasele reale ale obiectelor.
Coloanele sunt separate prin TAB.

# Bibliografie
<a name="CL" href="https://link.springer.com/article/10.1007%2Fs40745-015-0040-1">[1] *Xu, D., Tian, Y. A Comprehensive Survey of Clustering Algorithms. Ann. Data. Sci. 2, 165–193 (2015)*</a>